# 1장 introduction
## 교재
* 트랜스포머를 활용한 자연어 처리 : 허깅페이스 개발팀이 알려주는 자연어 애플리케이션 구축
* pipeline api 써서 모델 이용 

## 저자 github
* https://github.com/rickiepark/nlp-with-transformers

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/nlp-with-transformers/blob/main/01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [1]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리.    # local에서 실행하면 각 패키지의 버전을 맞춰줘야하기 때문에 가상환경설정을 해야함.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=1)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 600 (delta 12), reused 2 (delta 1), pack-reused 569
Receiving objects: 100% (600/600), 57.83 MiB | 26.07 MiB/s, done.
Resolving deltas: 100% (300/300), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.32.1
Using datasets v2.14.4
Using accelerate v0.22.0
Using sentencepiece v0.1.99
Using sacremoses v0.0.41


# 트랜스포머스 소개

<img alt="transformer-timeline" caption="The transformers timeline" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_timeline.png?raw=1" id="transformer-timeline"/>

## 인코더-디코더 프레임워크

<img alt="rnn" caption="Unrolling an RNN in time." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_rnn.png?raw=1" id="rnn"/>

<img alt="enc-dec" caption="Encoder-decoder architecture with a pair of RNNs. In general, there are many more recurrent layers than those shown." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_enc-dec.png?raw=1" id="enc-dec"/>

## 어텐션 메커니즘

<img alt="enc-dec-attn" caption="Encoder-decoder architecture with an attention mechanism for a pair of RNNs." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_enc-dec-attn.png?raw=1" id="enc-dec-attn"/>

<img alt="attention-alignment" width="500" caption="RNN encoder-decoder alignment of words in English and the generated translation in French (courtesy of Dzmitry Bahdanau)." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter02_attention-alignment.png?raw=1" id="attention-alignment"/>

<img alt="transformer-self-attn" caption="Encoder-decoder architecture of the original Transformer." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_self-attention.png?raw=1" id="transformer-self-attn"/>

## NLP의 전이 학습

<img alt="transfer-learning" caption="Comparison of traditional supervised learning (left) and transfer learning (right)." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_transfer-learning.png?raw=1" id="transfer-learning"/>  

<img alt="ulmfit" width="500" caption="The ULMFiT process (courtesy of Jeremy Howard)." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_ulmfit.png?raw=1" id="ulmfit"/>

## 허깅 페이스 트랜스포머스

## 트랜스포머 애플리케이션 둘러보기

In [2]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### 텍스트 분류

In [5]:
from transformers import pipeline
classifier = pipeline('text-classification')

In [6]:
import pandas as pd
outputs = classifier(text)
pd.DataFrame(outputs)     # 부정적감정 확률 90%

,label,score
0,NEGATIVE,0.901546


### 개체명 인식

In [7]:
ner_tagger = pipeline('ner', aggregation_strategy = 'simple')    # max, average
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669691,Decept,253,259
6,MISC,0.498350,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### 질문 답변

In [9]:
reader = pipeline('question-answering')
question = 'What does the customer want?'
outputs = reader(question = question, context = text)
pd.DataFrame([outputs])   # 단어(entity_group)가 위치하는 값?

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### 요약

In [12]:
summarizer = pipeline('summarization')
outputs = summarizer(text, max_length=60, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead. As a lifelong enemy
of the Decepticons, I hope you can understand


### 번역

In [15]:
translator = pipeline('translation_en_to_de', model = 'Helsinki-NLP/opus-mt-en-de')      # 영어를 독일어로 번역
outputs = translator(text, clean_up_tokenization_spaces = True, min_length = 100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus
Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete,
entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von
Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich
hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere
einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt.
Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte,
von Ihnen bald zu hören. Aufrichtig, Bumblebee.


### 텍스트 생성

In [17]:
from transformers import set_seed
set_seed(42)            #동일 결과를 재현하기 위해 지정

In [18]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. You have
requested a new Transformers 2 action figure and the Optimus Prime action figure
I received on June 17 was not a part of this purchase. After receiving your
gift, I have decided to call and request a new Transformers 2 action figure, or
even a new Hasbro action figure. I can confirm that in order to resolve these
issues,


## 허깅 페이스 생태계

<img alt="ecosystem" width="500" caption="An overview of the Hugging Face ecosystem of libraries and the Hub." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_hf-ecosystem.png?raw=1" id="ecosystem"/>

### 허깅 페이스 허브
* https://huggingface.co/
* models 확인 : 위의 페이지에서 Models > 특정 모델 선택 > compute
* language document: 위의 페이지에서 language 선택
* transformer document: 위의 페이지에서 Docs 선택 > Trnasformer

<img alt="hub-overview" width="1000" caption="The models page of the Hugging Face Hub, showing filters on the left and a list of models on the right." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_hub-overview.png?raw=1" id="hub-overview"/>

<img alt="hub-model-card" width="1000" caption="A example model card from the Hugging Face Hub. The inference widget is shown on the right, where you can interact with the model." src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter01_hub-model-card.png?raw=1" id="hub-model-card"/>

### 허깅 페이스 토크나이저

### 허깅 페이스 데이터셋

### 허깅 페이스 액셀러레이트

## 트랜스포머의 주요 도전 과제

## 결론